In [1]:
import os
import csv
import json
from IPython import embed
import scipy
import pickle as pkl
import numpy as np
import gensim
from progiter import ProgIter
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
phrases = [['fever'], ['dry', 'cough'], ['cough'], ['sore', 'throat'], ['malaise'],
          ['headache'], ['muscle', 'pain'], ['nasal', 'congestion'], ['shortness', 'of', 'breath'],
          ['dyspnea'], ['respiratory', 'distress'], ['tachypnea'], ['hypoxia'], ['bilateral', 'opacities'],
          ['lung', 'infiltrates'], ['acute', 'respiratory', 'distress', 'syndrome'], ['ARDS'], ['pneumonia']]

In [ ]:
pbm_pmc_model = KeyedVectors.load_word2vec_format('embeddings/w2v/PubMed-and-PMC-w2v.bin', binary=True)
wiki_pbm_pmc_model = KeyedVectors.load_word2vec_format('embeddings/w2v/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
bioasq_model = KeyedVectors.load_word2vec_format('embeddings/word2vecTools/w2v.txt')
biow2v_model = KeyedVectors.load_word2vec_format('embeddings/biow2v/BioWordVec_PubMed_MIMICIII_d200.vec.bin', 
                                                binary=True)
w2v300_model = Word2Vec.load('embeddings/clinical_embeddings/W2V_300/w2v_OA_CR_300d.bin')
extrinsic_model = KeyedVectors.load_word2vec_format('embeddings/biow2v/bio_embedding_extrinsic', binary=True)
intrinsic_model = KeyedVectors.load_word2vec_format('embeddings/biow2v/bio_embedding_intrinsic', binary=True)
standard_glove = KeyedVectors.load_word2vec_format('/sandata/sohamp/glove/glove.840B.300d.txt')

In [ ]:
models = [pbm_pmc_model, wiki_pbm_pmc_model, bioasq_model, w2v300_model, 
          extrinsic_model, intrinsic_model, standard_glove]

In [ ]:
all_returned_phrases = dict()
for phrase in phrases:
    returned_phrases = list()
    for model in models:
        results = list()
        try:
            results = model.most_similar(positive=phrase, topn=20)
        except:
            results = model.most_similar(positive=[x.lower() for x in phrase], topn=20)
        returned_phrases.append(results)
    all_returned_phrases[' '.join(phrase)] = returned_phrases